In [1]:
import os
import time
import warnings
import numpy as np
import progressbar
import requests
import pandas as pd
from pathlib import Path
from tqdm import tqdm
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup


url = "https://github.com/onnx/models"

response = requests.get(url, verify=False)
soup = BeautifulSoup(response.text, 'html.parser')
tables = soup.findAll('table')

model_df_list = []
for table in tables:
    records = []
    columns = []
    for tr in table.findAll("tr"):
        ths = tr.findAll("th")
        if ths != []:
            for each in ths:
                columns.append(each.text)
        else:
            trs = tr.findAll("td")
            record = []
            for each in trs:
                try:
                    link = each.find('a')['href']
                    text = each.text
                    record.append((link, text))
                except:
                    text = each.text
                    record.append(('', text))
            records.append(record)
    df = pd.DataFrame(data=records, columns = columns)
    model_df_list.append(df)

In [2]:
df = model_df_list[0]
df

,Model Class,Reference,Description,Huggingface Spaces
0,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1801.04381, Sandler et ...","(, Light-weight deep neural network best suite...","(, )"
1,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1512.03385, He et al.)","(, A CNN model (up to 152 layers). Uses shortc...","(https://huggingface.co/spaces/onnx/ResNet, )"
2,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1602.07360, Iandola et ...","(, A light-weight CNN model providing AlexNet ...","(https://huggingface.co/spaces/onnx/SqueezeNet, )"
3,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1409.1556, Simonyan et ...","(, Deep CNN model(up to 19 layers). Similar to...","(https://huggingface.co/spaces/onnx/VGG, )"
4,(/onnx/models/blob/main/vision/classification/...,(https://papers.nips.cc/paper/4824-imagenet-cl...,"(, A Deep CNN model (up to 8 layers) where the...","(https://huggingface.co/spaces/onnx/AlexNet, )"
5,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/pdf/1409.4842.pdf, Szegedy ...","(, Deep CNN model(up to 22 layers). Comparativ...","(https://huggingface.co/spaces/onnx/GoogleNet, )"
6,(/onnx/models/blob/main/vision/classification/...,(https://ucb-icsi-vision-group.github.io/caffe...,"(, Deep CNN variation of AlexNet for Image Cla...","(https://huggingface.co/spaces/onnx/CaffeNet, )"
7,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1311.2524, Girshick et ...","(, Pure Caffe implementation of R-CNN for imag...","(, )"
8,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1608.06993, Huang et al.)","(, Model that has every layer connected to eve...",(https://huggingface.co/spaces/onnx/DenseNet-1...
9,(/onnx/models/blob/main/vision/classification/...,"(https://arxiv.org/abs/1409.4842, Szegedy et al.)","(, This model is same as GoogLeNet, implemente...",(https://huggingface.co/spaces/onnx/Inception_...


In [3]:
download_url_list = []
try:
    for df in model_df_list:
        for link, text in df['Model Class']:
            link_url = f'https://github.com{link}'
            response = requests.get(link_url, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
            model_tables = soup.findAll('table')
            
            for model_table in model_tables:
                records = []
                columns = []
                for tr in model_table.findAll("tr"):
                    ths = tr.findAll("th")
                    if ths != []:
                        for each in ths:
                            columns.append(each.text)
                    else:
                        trs = tr.findAll("td")
                        record = []
                        for each in trs:
                            try:
                                link = each.find('a')['href']
                                text = each.text
                                record.append((link, text))
                            except:
                                text = each.text
                                record.append(('', text))
                        records.append(record)
                df_model = pd.DataFrame(data=records, columns = columns)
                for link, _ in df_model.get("Download", default=[]):
                    download_url = f'https://github.com{link}?raw=true'
                    download_url_list.append(download_url)
except Exception as e:
    print(e)

In [4]:
def download_file(url, base_path=None, n_chunk=1024):
    if base_path:
        output_path = f'{base_path}/{Path(url).name.split("?")[0]}'
    else:
        output_path = f'{Path(url).name}'
    r = requests.get(url, stream=True, verify=False)
    # Estimates the number of bar updates
    block_size = 1024
    file_size = int(r.headers.get('Content-Length', None))
    num_bars = np.ceil(file_size / (n_chunk * block_size))
    bar =  progressbar.ProgressBar(maxval=num_bars).start()
    with open(output_path, 'wb') as f:
        for i, chunk in enumerate(r.iter_content(chunk_size=n_chunk * block_size)):
            f.write(chunk)
            bar.update(i+1)
            # Add a little sleep so you can see the bar progress
            time.sleep(0.05)

In [6]:
base_path = '/Users/raviraagavsr/Data/Onnx_Model_Zoo'

os.makedirs(base_path, exist_ok=True)

for url in tqdm(download_url_list, desc='Processing'):
    download_file(url, base_path=base_path)

Processing: 100%|██████████| 175/175 [3:38:49<00:00, 75.03s/it] ############# |                                                  |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        |  0% |                                                                        | 